In [1]:
import pyspark
sc = pyspark.SparkContext.getOrCreate()

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1600111618676_0003,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
moviesAndRates_DF = spark.read.csv("s3://movielens-dataset-bucket/moviesAndRates.csv", \
                                   header = True, \
                                   inferSchema = True, \
                                   ignoreLeadingWhiteSpace = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
moviesAndRates_DF.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+------+----+-------------------+
| ID|    Title|Year|(no genres listed)|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|IMAX|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|UserID|Rate|          Timestamp|
+---+---------+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+------+----+-------------------+
|  1|Toy Story|1995|                 0|     0|        1|        1|       1|     1|    0|          0|    0|      1|        0|     0|   0|      0|      0|      0|     0|       0|  0|      0|   122| 5.0|1996-08-02 11:24:06|
|  1|Toy Story|1995|                 0|     0|        1|        1|       1|     1|    0|          0|    0|      1|  

ARRRRGH! What a mess! Let's show the data in vertical view.

In [4]:
moviesAndRates_DF.show(3, vertical = True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0---------------------------------
 ID                 | 1                   
 Title              | Toy Story           
 Year               | 1995                
 (no genres listed) | 0                   
 Action             | 0                   
 Adventure          | 1                   
 Animation          | 1                   
 Children           | 1                   
 Comedy             | 1                   
 Crime              | 0                   
 Documentary        | 0                   
 Drama              | 0                   
 Fantasy            | 1                   
 Film-Noir          | 0                   
 Horror             | 0                   
 IMAX               | 0                   
 Musical            | 0                   
 Mystery            | 0                   
 Romance            | 0                   
 Sci-Fi             | 0                   
 Thriller           | 0                   
 War                | 0                   
 Western   

In order to use DF in SQL query, it has to be registered.

In [5]:
moviesAndRates_DF.createOrReplaceTempView("moviesAndRates")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
spark.sql("select * from moviesAndRates limit 2").show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---------+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+------+----+-------------------+
| ID|    Title|Year|(no genres listed)|Action|Adventure|Animation|Children|Comedy|Crime|Documentary|Drama|Fantasy|Film-Noir|Horror|IMAX|Musical|Mystery|Romance|Sci-Fi|Thriller|War|Western|UserID|Rate|          Timestamp|
+---+---------+----+------------------+------+---------+---------+--------+------+-----+-----------+-----+-------+---------+------+----+-------+-------+-------+------+--------+---+-------+------+----+-------------------+
|  1|Toy Story|1995|                 0|     0|        1|        1|       1|     1|    0|          0|    0|      1|        0|     0|   0|      0|      0|      0|     0|       0|  0|      0|   122| 5.0|1996-08-02 11:24:06|
|  1|Toy Story|1995|                 0|     0|        1|        1|       1|     1|    0|          0|    0|      1|  

### Top 10 movies (only those with more than 50 rates)

In [7]:
spark.sql("select Title, round(avg(Rate), 2) as Avg_Rate "
          "from moviesAndRates "
          "group by Title having count(Rate) > 50 "
          "order by Avg_Rate desc "
          "limit 10").show(truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------------------------------------------+--------+
|Title                                     |Avg_Rate|
+------------------------------------------+--------+
|Happy Weekend                             |4.89    |
|The Texas Chainsaw Massacre: The Beginning|4.82    |
|Hotel Rwanda                              |4.8     |
|Lola                                      |4.78    |
|Road to Perdition                         |4.77    |
|SSSSSSS                                   |4.77    |
|Tokyo Fist (Tokyo ken)                    |4.71    |
|The Swamp                                 |4.68    |
|Ariel                                     |4.68    |
|Happy Gilmore                             |4.68    |
+------------------------------------------+--------+

### Number of rates for each movie

In [8]:
query = ("select Title, count(Rate) as Count_Rates "
          "from moviesAndRates "
          "group by Title "
          "order by Count_Rates desc")

spark.sql(query).show(20, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------------------+-----------+
|Title                        |Count_Rates|
+-----------------------------+-----------+
|The Other Side of Sunday     |4165       |
|Dead End                     |3509       |
|Demonlover                   |3414       |
|Yu-Gi-Oh!                    |3164       |
|Character (Karakter)         |2529       |
|Morocco                      |2345       |
|Mean Machine                 |2337       |
|Formula 51                   |2152       |
|The Pickle                   |2113       |
|The General                  |2002       |
|The Sea Inside               |1969       |
|Zoot Suit                    |1914       |
|Hush... Hush. Sweet Charlotte|1833       |
|Salome's Last Dance          |1776       |
|For Sale (À Vendre)          |1731       |
|Lethal Weapon 4              |1729       |
|The Whole Nine Yards         |1727       |
|Everyone's Hero              |1722       |
|Allegro non Troppo           |1707       |
|Vacancy                      |1

### How many movies were assign to specific genre?

I think that here we don't need full tabel. We just need one row from each movie, so columns 'UserID, 'Rate, 'Timestamp' could be dropped and get distinct row for each movie.

I'll create another DF and register it.

In [9]:
columns = moviesAndRates_DF.columns
columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['ID', 'Title', 'Year', '(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western', 'UserID', 'Rate', 'Timestamp']

In [10]:
genres = [genre for genre in columns if genre not in ["ID", "Title", "Year", "UserID", "Rate", "Timestamp"]]
genres

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['(no genres listed)', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'IMAX', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [11]:
moviesAndRates_DF1 = moviesAndRates_DF.drop("UserID", "Rate", "Timestamp").drop_duplicates() \
    .withColumnRenamed("(no genres listed)", "No_Genre") \
    .withColumnRenamed("Film-Noir", "Film_Noir") \
    .withColumnRenamed("Sci-Fi", "SciFi") \

moviesAndRates_DF1.createOrReplaceTempView("moviesAndRates1")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
spark.sql("select sum(No_Genre) No_Genre, sum(Action) Action, sum(Adventure) Adventure, sum(Animation) Animation, sum(Children) Children, sum(Comedy) Comedy, sum(Crime) Crime, sum(Documentary) Documentary, sum(Drama) Drama, sum(Fantasy) Fantasy, sum(Film_Noir) Film_Noir, sum(Horror) Horror, sum(IMAX) IMAX, sum(Musical) Musical, sum(Mystery) Mystery, sum(Romance) Romance, sum(SciFi) SciFi, sum(Thriller) Thriller, sum(War) War, sum(Western) Western "
          "from moviesAndRates1").show(len(genres), vertical = True, truncate = False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-RECORD 0-----------
 No_Genre    | 1    
 Action      | 1438 
 Adventure   | 999  
 Animation   | 281  
 Children    | 507  
 Comedy      | 3609 
 Crime       | 1087 
 Documentary | 470  
 Drama       | 5204 
 Fantasy     | 530  
 Film_Noir   | 144  
 Horror      | 981  
 IMAX        | 28   
 Musical     | 429  
 Mystery     | 494  
 Romance     | 1645 
 SciFi       | 732  
 Thriller    | 1668 
 War         | 499  
 Western     | 272

### Number of movies for each year

In [13]:
years = moviesAndRates_DF1.select("Year").distinct()
years.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+
|Year|
+----+
|2003|
|1955|
|1978|
|1925|
|1944|
+----+
only showing top 5 rows

In [14]:
years_count = years.count()
years_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

94

In [15]:
spark.sql("select Year, count(ID) Movies from moviesAndRates1 group by Year order by Year asc").show(years_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+------+
|Year|Movies|
+----+------+
|1915|     1|
|1916|     2|
|1917|     2|
|1918|     2|
|1919|     4|
|1920|     5|
|1921|     3|
|1922|     7|
|1923|     6|
|1924|     6|
|1925|    10|
|1926|    10|
|1927|    19|
|1928|    10|
|1929|     7|
|1930|    15|
|1931|    13|
|1932|    20|
|1933|    23|
|1934|    16|
|1935|    18|
|1936|    30|
|1937|    29|
|1938|    19|
|1939|    37|
|1940|    37|
|1941|    27|
|1942|    37|
|1943|    40|
|1944|    36|
|1945|    35|
|1946|    37|
|1947|    38|
|1948|    44|
|1949|    37|
|1950|    44|
|1951|    44|
|1952|    40|
|1953|    52|
|1954|    43|
|1955|    56|
|1956|    52|
|1957|    61|
|1958|    61|
|1959|    61|
|1960|    64|
|1961|    55|
|1962|    68|
|1963|    62|
|1964|    68|
|1965|    72|
|1966|    87|
|1967|    67|
|1968|    70|
|1969|    62|
|1970|    66|
|1971|    72|
|1972|    79|
|1973|    77|
|1974|    73|
|1975|    71|
|1976|    74|
|1977|    81|
|1978|    81|
|1979|    86|
|1980|   160|
|1981|   173|
|1982|   166|
|1983|

Just for a comparision, I'll do the same with Python code.

In [16]:
moviesAndRates_DF.groupBy("Year").count().orderBy("Year", ascending = True).show(years_count)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----+-----+
|Year|count|
+----+-----+
|1915|  120|
|1916|  405|
|1917|  192|
|1918|   95|
|1919|  916|
|1920|  432|
|1921|  139|
|1922|  338|
|1923|  343|
|1924| 1271|
|1925| 1187|
|1926| 1669|
|1927| 5337|
|1928| 1090|
|1929|  683|
|1930| 4949|
|1931|  777|
|1932| 2855|
|1933| 4154|
|1934| 3071|
|1935| 1796|
|1936| 3436|
|1937| 3298|
|1938| 2562|
|1939| 4466|
|1940| 6106|
|1941| 2341|
|1942| 7262|
|1943| 5816|
|1944| 3028|
|1945| 6084|
|1946| 3973|
|1947| 5689|
|1948| 5941|
|1949| 5084|
|1950| 5389|
|1951| 6216|
|1952| 4683|
|1953| 4689|
|1954| 7914|
|1955| 6674|
|1956| 6612|
|1957| 7430|
|1958| 8981|
|1959| 9954|
|1960| 8007|
|1961| 9405|
|1962|10016|
|1963| 8350|
|1964| 9883|
|1965|11323|
|1966|12966|
|1967| 9021|
|1968| 7779|
|1969| 8512|
|1970| 8017|
|1971| 9782|
|1972|14517|
|1973|10188|
|1974| 8618|
|1975| 7733|
|1976| 7392|
|1977|11960|
|1978| 9694|
|1979|11543|
|1980|20911|
|1981|25920|
|1982|21420|
|1983|16176|
|1984|20362|
|1985|19044|
|1986|17578|
|1987|27054|
|1988|31314|

As you can see above, execution time for SQL was around 7 second and Python arount 4 seconds.